In [ ]:
from pathlib import Path
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

import pytorch_lightning as pl
from multipage_classifier.transformer_model import MultipageTransformerConfig
from training.transformer.lightning_module import MultipageTransformerPLModule, MultipagePLDataModule


In [ ]:

MODEL = "../lightning_logs/multipage_transformer/version_0/checkpoints/best-checkpoint.ckpt"

DATASET_PATH="../dataset/ucsf-idl-resized-without_emails/"
DS_FILE = "IDL-less_20_pages.json"

N_EPOCHS = 50
MAX_PAGES = 16

NUM_WORKERS = 1
 
MAX_LENGTH = 768
IMAGE_SIZE  = [640, 896]

In [ ]:
# Load Model

model = MultipageTransformerPLModule.load_from_checkpoint(MODEL)

In [ ]:
data_module = MultipagePLDataModule(
    DATASET_PATH,
    DS_FILE,
    model.model
)
data_module.prepare_data()
data_module.setup()

In [ ]:
i = next(iter(data_module.test_dataloader()))

In [ ]:
preds, answers, scores = model.validation_step(i)